In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import numpy as np
import cpi

In [2]:
#Scrape Senator salary overtime Wikipedia 
url = "https://en.wikipedia.org/wiki/Salaries_of_members_of_the_United_States_Congress"
response = requests.get(url)
response.raise_for_status() 

In [3]:
# Parse HTML content 

soup = BeautifulSoup(response.text, 'html.parser')

In [4]:
# Find the table
table = soup.find('table', class_='wikitable')

In [5]:
# Extract table headers
headers = [header.text.strip() for header in table.find_all('th')]

In [6]:
# Extract table rows
rows = []
for row in table.find_all('tr')[1:]:
    cells = row.find_all('td')
    if len(cells) == len(headers):
        rows.append([cell.text.strip() for cell in cells])

In [7]:
# Create DataFrame
df = pd.DataFrame(rows, columns=headers)
df

,Year,Salary,Per diem/annum,Auto COLA adj.,In 2023 dollars (when instituted),In 2023 dollars (year prior to next increase)
0,1789,$6,per diem,,$147,$118
1,1795,$7,per diem,,$121,$228
2,1855,"$3,000",per annum,,"$94,221","$56,132"
3,1865,"$5,000",per annum,,"$95,587","$115,711"
4,1871,"$7,500",per annum,,"$183,208","$183,208"
5,1874,"$5,000",per annum,,"$129,324","$162,852"
6,1907,"$7,500",per annum,,"$235,554","$128,068"
7,1925,"$10,000",per annum,,"$166,869","$192,429"
8,1935,"$10,000",per annum,,"$213,447","$150,068"
9,1947,"$12,500",per annum,,"$163,823","$136,214"


In [8]:
# Clean DataFrame
df['Year'] = df['Year'].str.replace(' (2/4)', '').astype(int)
df = df[df['Year'] > 1923]
df['Salary'] = df['Salary'].str.replace('$', '')
df['Salary'] = df['Salary'].str.replace(',', '').astype(float)
df = df[['Year', 'Salary' ]]

df

C:\Users\Janea\AppData\Local\Temp\ipykernel_14664\2322947748.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Salary'] = df['Salary'].str.replace('$', '')
C:\Users\Janea\AppData\Local\Temp\ipykernel_14664\2322947748.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Salary'] = df['Salary'].str.replace(',', '').astype(float)


,Year,Salary
7,1925,10000.0
8,1935,10000.0
9,1947,12500.0
10,1955,22500.0
11,1965,30000.0
12,1969,42500.0
13,1975,44600.0
14,1977,57500.0
15,1979,60652.5
16,1985,75100.0


In [9]:
# import min wage data
df_min_wage = pd.read_csv(r"C:\Users\Janea\Downloads\min_wage.csv").dropna()
df_min_wage['Year'] = df_min_wage['year'].astype(int)
df_min_wage = df_min_wage.drop(columns='year')
df_min_wage

,min_wage,Year
0,0.26,1939
1,0.30,1940
2,0.30,1941
3,0.30,1942
4,0.30,1943
...,...,...
80,7.25,2019
81,7.25,2020
82,7.25,2021
83,7.25,2022


In [10]:
# Merge Min Wage Data with Senate Salary Data
df_combined = pd.merge(df, df_min_wage, how = 'outer', on = 'Year')
df_combined

,Year,Salary,min_wage
0,1925,10000.0,NaN
1,1935,10000.0,NaN
2,1939,NaN,0.26
3,1940,NaN,0.30
4,1941,NaN,0.30
...,...,...,...
82,2019,NaN,7.25
83,2020,174000.0,7.25
84,2021,NaN,7.25
85,2022,NaN,7.25


In [11]:
# Fill in missing salaries
df_combined['Salary'] = df_combined['Salary'].ffill()
df_combined

,Year,Salary,min_wage
0,1925,10000.0,NaN
1,1935,10000.0,NaN
2,1939,10000.0,0.26
3,1940,10000.0,0.30
4,1941,10000.0,0.30
...,...,...,...
82,2019,174000.0,7.25
83,2020,174000.0,7.25
84,2021,174000.0,7.25
85,2022,174000.0,7.25


In [12]:
# use CPI package to generate to adj for inflation
df_combined['Adjusted Salary'] = df_combined.apply(lambda x: cpi.inflate(x['Salary'], int(x['Year'])), axis=1).astype(int)
df_combined


,Year,Salary,min_wage,Adjusted Salary
0,1925,10000.0,NaN,174115
1,1935,10000.0,NaN,222410
2,1939,10000.0,0.26,219210
3,1940,10000.0,0.30,217644
4,1941,10000.0,0.30,207280
...,...,...,...,...
82,2019,174000.0,7.25,207379
83,2020,174000.0,7.25,204852
84,2021,174000.0,7.25,195660
85,2022,174000.0,7.25,181162


In [13]:
# Convert min wage to annual salary (assuming 2080 hours worked per year)
df_combined['min_wage'] = df_combined['min_wage'].fillna('0')
df_combined['min_wage'] = df_combined['min_wage'].astype(float)
df_combined['min_wage_annual'] = df_combined['min_wage']*2080
df_combined

,Year,Salary,min_wage,Adjusted Salary,min_wage_annual
0,1925,10000.0,0.00,174115,0.0
1,1935,10000.0,0.00,222410,0.0
2,1939,10000.0,0.26,219210,540.8
3,1940,10000.0,0.30,217644,624.0
4,1941,10000.0,0.30,207280,624.0
...,...,...,...,...,...
82,2019,174000.0,7.25,207379,15080.0
83,2020,174000.0,7.25,204852,15080.0
84,2021,174000.0,7.25,195660,15080.0
85,2022,174000.0,7.25,181162,15080.0


In [16]:
# Adjust minimum wage annual for inflation
df_combined['Adjusted Minimum Wage Annual'] = df_combined.apply(lambda x: cpi.inflate(x['min_wage_annual'], int(x['Year'])), axis=1).astype(int)
df_combined

,Year,Salary,min_wage,Adjusted Salary,min_wage_annual,Adjusted Minimum Wage Annual
0,1925,10000.0,0.00,174115,0.0,0
1,1935,10000.0,0.00,222410,0.0,0
2,1939,10000.0,0.26,219210,540.8,11854
3,1940,10000.0,0.30,217644,624.0,13581
4,1941,10000.0,0.30,207280,624.0,12934
...,...,...,...,...,...,...
82,2019,174000.0,7.25,207379,15080.0,17972
83,2020,174000.0,7.25,204852,15080.0,17753
84,2021,174000.0,7.25,195660,15080.0,16957
85,2022,174000.0,7.25,181162,15080.0,15700


In [17]:
df_combined = df_combined.rename(columns = {'Adjusted Salary':'Senator Adj. Salary'})


In [18]:
df_combined

,Year,Salary,min_wage,Senator Adj. Salary,min_wage_annual,Adjusted Minimum Wage Annual
0,1925,10000.0,0.00,174115,0.0,0
1,1935,10000.0,0.00,222410,0.0,0
2,1939,10000.0,0.26,219210,540.8,11854
3,1940,10000.0,0.30,217644,624.0,13581
4,1941,10000.0,0.30,207280,624.0,12934
...,...,...,...,...,...,...
82,2019,174000.0,7.25,207379,15080.0,17972
83,2020,174000.0,7.25,204852,15080.0,17753
84,2021,174000.0,7.25,195660,15080.0,16957
85,2022,174000.0,7.25,181162,15080.0,15700


In [23]:
df_combined.to_csv('senator_salary_data.csv', index=False)

In [22]:
df.to_csv('adjusted.csv', index=False)